In [1]:
import pandas as pd

In [2]:
stations = pd.read_csv("stations_table.csv")
stations = stations[['표준정류장ID','이비카드정류장ID','관할관청','WGS84위도','WGS84경도']]
stations.columns = ['표준정류장ID', '이비카드정류장ID', '관할관청','위도','경도']

In [3]:
stations['표준정류장ID'].isna().sum()
stations['이비카드정류장ID'].isna().sum()

12542

In [4]:
stations.dropna(subset=["이비카드정류장ID"],inplace = True)

In [5]:
combined = pd.read_csv("routestationinfo.csv")
combined.drop(combined.columns[0],axis=1)

,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no
0,228000018,10-4,1,용인터미널,228001552,47634.0
1,228000018,10-4,2,용인터미널(경유),277102443,NaN
2,228000018,10-4,3,포브스병원,228000443,29439.0
3,228000018,10-4,4,제일교회,228000665,29881.0
4,228000018,10-4,5,라이프아파트,228000664,29457.0
...,...,...,...,...,...,...
37826,241491011,38-1,63,신일해피트리후문,233002733,NaN
37827,241491011,38-1,64,기배동행정복지센터,233002726,NaN
37828,241491011,38-1,65,기안1통입구,233002959,NaN
37829,241491011,38-1,66,배양2리,233002717,55455.0


In [6]:
combined = combined[['bus_line_no', 'station_nm', 'station_id']]
combined.columns = ['노선번호', '정류장이름', '표준정류장ID']

In [7]:
re_combined = pd.merge(combined, stations, on = "표준정류장ID")
re_combined = re_combined[re_combined['관할관청'] == '경기도 화성시']

In [8]:
re_combined = re_combined.drop(re_combined.columns[0], axis = 1)
final_combined = re_combined.drop_duplicates()
final_combined.head()

,정류장이름,표준정류장ID,이비카드정류장ID,관할관청,위도,경도
1522,한원IC,233000169,4170725.0,경기도 화성시,37.152733,127.122883
1523,장지동,233000168,4170397.0,경기도 화성시,37.155117,127.117900
1526,장지동,233000167,4170396.0,경기도 화성시,37.155867,127.115017
1529,꽃가마약수터,233000166,4170395.0,경기도 화성시,37.159100,127.099350
1607,꽃가마약수터,233001323,4170651.0,경기도 화성시,37.158900,127.098750


In [9]:
tripchain = pd.read_csv("TripChain.csv", encoding = "utf-8")

In [10]:
tripchain.columns

Index(['암호화카드번호||'', ''||트랜잭션ID||'', ''||환승횟수||'', ''||교통카드발행사ID||'',
       ''||총이용객수||'', ''||사용자구분||'', ''||교통수단CD1||'', ''||교통수단CD2||'',
       ''||교통수단CD3||'', ''||교통수단CD4||'', ''||교통수단CD5||'', ''||버스노선ID1||'',
       ''||버스노선ID2||'', ''||버스노선ID3||'', ''||버스노선ID4||'', ''||버스노선ID5||'',
       ''||차량ID1||'', ''||차량ID2||'', ''||차량ID3||'', ''||차량ID4||'',
       ''||차량ID5||'', ''||총통행거리||'', ''||총탑승시간||'', ''||총소요시간||'',
       ''||승차일시1||'', ''||승차일시2||'', ''||승차일시3||'', ''||승차일시4||'',
       ''||승차일시5||'', ''||하차일시1||'', ''||하차일시2||'', ''||하차일시3||'',
       ''||하차일시4||'', ''||하차일시5||'', ''||최초승차일시||'', ''||최종하차일시||'',
       ''||승차역ID1||'', ''||승차역ID2||'', ''||승차역ID3||'', ''||승차역ID4||'',
       ''||승차역ID5||'', ''||하차역ID1||'', ''||하차역ID2||'', ''||하차역ID3||'',
       ''||하차역ID4||'', ''||하차역ID5||'', ''||최초승차역ID||'', ''||최종하차역ID||'',
       ''||총이용금액||'', ''||수집건수||'', ''||트립체인완료코드'],
      dtype='object')

In [11]:
tripchain = tripchain[["'||사용자구분||'","'||최초승차일시||'","'||최종하차일시||'","'||승차역ID1||'","'||최종하차역ID||'"]]

In [12]:
tripchain.columns = ['사용자구분','최초승차일시','최종하차일시','최초승차역ID','최종하차역ID']

In [13]:
to_str = tripchain['최초승차일시'].astype(str)
to_df = pd.DataFrame(to_str)
to_dt = pd.to_datetime(to_df['최초승차일시'], errors = 'coerce')
dt_to_df = pd.DataFrame(to_dt)
tripchain['최초승차일시'] = dt_to_df

In [14]:
tripchain['최종하차일시'] = pd.to_datetime(tripchain['최종하차일시'], errors = 'coerce')

In [15]:
tripchain['승차시간대'] = tripchain['최초승차일시'].dt.strftime('%H')
tripchain['하차시간대'] = tripchain['최종하차일시'].dt.strftime('%H')

In [16]:
morning = ['07','08','09']
evening = ['18','19','20']
df_morning = tripchain[tripchain['승차시간대'].isin(morning)]
df_evening = tripchain[tripchain['하차시간대'].isin(evening)]
df_morning.head()

,사용자구분,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,승차시간대,하차시간대
1,1,2018-07-01 07:21:56,2018-07-01 07:25:20,4117280.0,4117269.0,07,07
4,1,2018-07-01 08:50:58,2018-07-01 08:59:10,4100122.0,4116848.0,08,08
6,1,2018-07-01 07:57:31,2018-07-01 08:53:59,4197606.0,4116627.0,07,08
10,1,2018-07-01 09:10:18,2018-07-01 09:32:31,4116704.0,4151749.0,09,09
23,1,2018-07-01 09:34:24,2018-07-01 09:49:49,4198644.0,4108007.0,09,09


In [17]:
df_morning_on = df_morning[['사용자구분','최초승차일시','최초승차역ID','승차시간대']]
df_morning_off = df_morning[['사용자구분','최종하차일시','최종하차역ID','하차시간대']]

In [18]:
df_evening_on = df_evening[['사용자구분','최초승차일시','최초승차역ID','승차시간대']]
df_evening_off = df_evening[['사용자구분','최종하차일시','최종하차역ID','하차시간대']]

In [19]:
df_morning_on.rename(columns ={'최초승차역ID':'이비카드정류장ID'}, inplace = True)
df_morning_off.rename(columns ={'최종하차역ID':'이비카드정류장ID'}, inplace = True)

C:\Users\Yeonjae Hwang\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [20]:
df_79_on = pd.merge(df_morning_on, final_combined, on = '이비카드정류장ID')
df_79_off = pd.merge(df_morning_off, final_combined, on = "이비카드정류장ID")

In [21]:
df_evening_on.rename(columns ={'최초승차역ID':'이비카드정류장ID'}, inplace = True)
df_evening_off.rename(columns ={'최종하차역ID':'이비카드정류장ID'}, inplace = True)

In [38]:
df_1820_on = pd.merge(df_evening_on, final_combined, on = '이비카드정류장ID')
df_1820_off = pd.merge(df_evening_off, final_combined, on = '이비카드정류장ID')

In [39]:
from collections import Counter

In [40]:
df_79_on_count = Counter(df_79_on['정류장이름'])
mc_79_on = df_79_on_count.most_common(50)
df_mc_79_on = pd.DataFrame(mc_79_on)
df_mc_79_on.columns = ['정류장이름','횟수']
df_mc_79_on.head()

,정류장이름,횟수
0,신영통현대타운.두산위브,1564
1,반월리큰고개,996
2,신창미션힐.송화초교,857
3,다은마을(중),813
4,와우2리,792


In [41]:
point_79_on = pd.merge(df_79_on[['이비카드정류장ID','정류장이름','위도','경도']], df_mc_79_on, on = '정류장이름', how = 'outer')

In [75]:
point_79_on = point_79_on.drop_duplicates(['정류장이름'], keep = 'last')
point_79_on = point_79_on.dropna()

In [77]:
point_79_on.sort_values(['횟수'], ascending=[False]).head()

,이비카드정류장ID,정류장이름,위도,경도,횟수
17935,4108037.0,신영통현대타운.두산위브,37.235333,127.062100,1564.0
6926,4108035.0,반월리큰고개,37.232550,127.064783,996.0
4346,4100048.0,신창미션힐.송화초교,37.203300,127.038550,857.0
9735,4195716.0,다은마을(중),37.200417,127.067300,813.0
14550,4116673.0,와우2리,37.215900,126.976183,792.0


In [57]:
df_79_off_count = Counter(df_79_off['정류장이름'])
mc_79_off = df_79_off_count.most_common(50)
df_mc_79_off = pd.DataFrame(mc_79_off)
df_mc_79_off.columns = ['정류장이름','횟수']

In [45]:
point_79_off = pd.merge(df_79_off[['이비카드정류장ID','정류장이름','위도','경도']], df_mc_79_off, on = '정류장이름', how = 'outer')

In [69]:
point_79_off = point_79_off.drop_duplicates(['정류장이름'], keep = 'last')
point_79_off = point_79_off.dropna()

In [71]:
point_79_off.sort_values(['횟수'], ascending=[False]).head()

,이비카드정류장ID,정류장이름,위도,경도,횟수
20044,4197592.0,IT단지(중),37.224717,127.074183,2387.0
13639,4105628.0,삼성반도체후문,37.220933,127.075783,1618.0
11050,4199455.0,한림대병원(중),37.216633,127.078117,1375.0
1744,4117226.0,수원대학교,37.213750,126.979967,964.0
14877,4170973.0,메타폴리스(중),37.203700,127.067500,637.0


In [48]:
df_1820_on_count = Counter(df_1820_on['정류장이름'])
mc_1820_on = df_1820_on_count.most_common(50)
df_mc_1820_on = pd.DataFrame(mc_1820_on)
df_mc_1820_on.columns = ['정류장이름','횟수']
point_1820_on = pd.merge(df_1820_on[['이비카드정류장ID','정류장이름','위도','경도']], df_mc_1820_on, on = '정류장이름', how = 'outer')

In [67]:
point_1820_on = point_1820_on.drop_duplicates(['정류장이름'], keep = 'last')
point_1820_on = point_1820_on.dropna()

In [73]:
point_1820_on.sort_values(['횟수'], ascending=[False]).head()

,이비카드정류장ID,정류장이름,위도,경도,횟수
15041,4130123.0,삼성반도체후문,37.227667,127.070683,1604.0
3531,4197593.0,IT단지(중),37.225267,127.073483,1503.0
11543,4170977.0,한림대병원(중),37.217317,127.077750,1215.0
20966,4117227.0,수원대학교,37.213750,126.980150,1031.0
4882,4170973.0,메타폴리스(중),37.203700,127.067500,880.0


In [51]:
df_1820_off_count = Counter(df_1820_off['정류장이름'])
mc_1820_off = df_1820_off_count.most_common(50)
df_mc_1820_off = pd.DataFrame(mc_1820_off)
df_mc_1820_off.columns = ['정류장이름','횟수']

In [52]:
point_1820_off = pd.merge(df_1820_off[['이비카드정류장ID','정류장이름','위도','경도']], df_mc_1820_off, on = '정류장이름', how = 'outer')

In [63]:
point_1820_off = point_1820_off.drop_duplicates(['정류장이름'], keep = 'last')
point_1820_off = point_1820_off.dropna()

In [74]:
point_1820_off.sort_values(['횟수'], ascending=[False]).head()

,이비카드정류장ID,정류장이름,위도,경도,횟수
3987,4108036.0,신영통현대타운.두산위브,37.235467,127.062467,1276.0
7412,4170973.0,메타폴리스(중),37.203700,127.067500,795.0
8388,4170243.0,동탄1동행정복지센터,37.206650,127.072567,772.0
879,4108034.0,반월리큰고개,37.233000,127.064800,763.0
17270,4199835.0,능동마을입구,37.216850,127.058150,671.0


In [78]:
point_79_on.to_csv("point_79_on.csv", encoding = 'utf-8')
point_79_off.to_csv("point_79_off.csv", encoding = 'utf-8')
point_1820_on.to_csv("point_1820_on.csv", encoding = 'utf-8')
point_1820_off.to_csv("point_1820_off.csv", encoding = 'utf-8')